# Просмотр данных

## Импорт библиотек и исходного датасета, получение предварительной информации

Импортируем необходимые для работы библиотеки

In [1]:
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px
import optuna
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, recall_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

Рассмотрим первые 5 строк исходного датасета

In [2]:
data_initial = pd.read_csv('heart.csv.xls')
data_initial_cb = pd.read_csv('heart.csv.xls')

data_initial.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


Для более удобной ориентации по столбцам создадим небольшой глоссарий, где укажем перевод столбцов на русский язык.

1. `Age` - возраст.
2. `Sex` - пол.
3. `ChestPainType` - тип боли в груди.
4. `RestingBP` - артериальное давжение в состоянии покоя.
5. `Cholesterol` - холестерин.
6. `FastingBS` - уровень сахара в крови натощак.
7. `RestingECG` - ЭКГ в состоянии покоя.
8. `MaxHR` - максимальная частота пульса.
9. `ExcerciseAngina` - стенокардия, вызванная физической нагрузкой.
10. `Oldpeak` - 
11. `ST_Slope` - 
12. `HeartDisease` - заболевание сердца.

Для дальнейшего анализа данных напишем функцию, которая выведет следующую информацио о датасете:
1. название столбца;
2. тип данных, содержащихся в столбце; 
3. количество заполненных строк в столбце; 
4. количество пропусков в столбце и их долю от общего числа значений; 
5. минимальное и максимальное значения в столбце;
6. можно ли отнести значения в столбце к булеву типу данных;
7. матрицу корреляции для датасета.

Для того, чтобы включить столбцы `Sex` и `ExcerciseAngina` в матрицу корреляции, заменим заменим значения на 0 и 1 (так как в каждом из столбцов один вариант исключает другой)

In [3]:
data_initial['ExerciseAngina'] = data_initial['ExerciseAngina'].replace('Y',1)
data_initial['ExerciseAngina'] = data_initial['ExerciseAngina'].replace('N',0)

In [4]:
data_initial['Sex'] = data_initial['Sex'].replace('M',1)
data_initial['Sex'] = data_initial['Sex'].replace('F',0)

In [5]:
def introduction_to_data_plotly(df):
    parameters = dict()
    parameters['column name'] = list(df.columns)
    parameters['type'] = list(df.dtypes)
    parameters['non-null'] = list(df.count() - df.isna().sum())
    parameters['null_quantity'] = list(df.isna().sum())
    parameters['null_percentage'] = list((df.isna().sum()/df.count())*100)
    parameters['min'] = list(df.min())
    parameters['max'] = list(df.max())
    
    info = pd.DataFrame(parameters)
    info.loc[(info['max'] == 1) & (info['min'] == 0), 'is_bool'] = 'bool'
    info.loc[(info['type'] == 'object'), 'max'] = '-'
    info.loc[(info['type'] == 'object'), 'min'] = '-'
    info.loc[(info['type'] == 'object') | (info['max'] != 1) | (info['min'] != 0), 'is_bool'] = '-'
    
    corr = df.corr()
    fig = ff.create_annotated_heatmap(
        z=corr.to_numpy().round(2),
        x=list(corr.columns.values),
        y=list(corr.columns.values),       
        xgap=3, ygap=3,
        zmin=-1, zmax=1,
        colorscale='ylgnbu',
        colorbar_thickness=30,
        colorbar_ticklen=3,
    )
    fig.update_layout(title_text='Correlation Matrix<b>',
                    title_x=0.5,
                    titlefont={'size': 24},
                    width=600, height=550,
                    xaxis_showgrid=False,
                    xaxis={'side': 'bottom'},
                    yaxis_showgrid=False,
                    yaxis_autorange='reversed',                   
                    paper_bgcolor=None,
                    )

    return  display(info), fig.show()

In [6]:
info_plotly = introduction_to_data_plotly(data_initial)

info_plotly

,column name,type,non-null,null_quantity,null_percentage,min,max,is_bool
0,Age,int64,918,0,0.0,28,77,-
1,Sex,int64,918,0,0.0,0,1,bool
2,ChestPainType,object,918,0,0.0,-,-,-
3,RestingBP,int64,918,0,0.0,0,200,-
4,Cholesterol,int64,918,0,0.0,0,603,-
5,FastingBS,int64,918,0,0.0,0,1,bool
6,RestingECG,object,918,0,0.0,-,-,-
7,MaxHR,int64,918,0,0.0,60,202,-
8,ExerciseAngina,int64,918,0,0.0,0,1,bool
9,Oldpeak,float64,918,0,0.0,-2.6,6.2,-


(None, None)

## Графическое представление данных

In [7]:
fig_cholesterol = ff.create_distplot(
    [data_initial.loc[data_initial['HeartDisease']==0]['Cholesterol'],data_initial.loc[data_initial['HeartDisease']==1]['Cholesterol']], 
    group_labels=['Without HeartDisease', 'With HeartDisease'],
    bin_size=20,
    curve_type='normal',
    colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']
)

fig_cholesterol.update_xaxes(title_text='Количество пациентов')
fig_cholesterol.update_yaxes(title_text='Плотность')
fig_cholesterol.update_layout(title_text='Гистограмма плотности распределения пациентов в зависимости от уровня холестерина')

fig_cholesterol.show()

In [8]:
fig_age = ff.create_distplot(
    [data_initial.loc[data_initial['HeartDisease']==0]['Age'],data_initial.loc[data_initial['HeartDisease']==1]['Age']], 
    group_labels=['Without HeartDisease', 'With HeartDisease'],
    bin_size=5,
    curve_type='normal',
    colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']
)

fig_age.update_xaxes(title_text='Возраст пациентов')
fig_age.update_yaxes(title_text='Плотность')
fig_age.update_layout(title_text='Гистограмма плотности распределения пациентов в зависимости от возраста')

fig_age.show()

In [9]:
fig_rest = ff.create_distplot(
    [data_initial.loc[data_initial['HeartDisease']==0]['RestingBP'],data_initial.loc[data_initial['HeartDisease']==1]['RestingBP']], 
    group_labels=['Without HeartDisease', 'With HeartDisease'],
    bin_size=10,
    curve_type='normal',
    colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']
)

fig_rest.update_xaxes(title_text='Уровень АД')
fig_rest.update_yaxes(title_text='Плотность')
fig_rest.update_layout(title_text='Гистограмма плотности распределения пациентов в зависимости от АД в состоянии покоя')

fig_rest.show()

In [10]:
fig_max = ff.create_distplot(
    [data_initial.loc[data_initial['HeartDisease']==0]['MaxHR'],data_initial.loc[data_initial['HeartDisease']==1]['MaxHR']], 
    group_labels=['Without HeartDisease', 'With HeartDisease'],
    bin_size=10,
    curve_type='normal',
    colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']
)

fig_max.update_xaxes(title_text='Максимальная частота пульса')
fig_max.update_yaxes(title_text='Плотность')
fig_max.update_layout(title_text='Гистограмма плотности распределения пациентов в зависимости от максимальной частоты пулься')

fig_max.show()

## Баланс классов

In [11]:
fig_heart = px.histogram(data_initial_cb, 
    x="HeartDisease",
    labels={'HeartDisease':'Наличие сердечно-сосудистой недостаточности', 'count':'Количество пациентов'},
    title="Пациенты с наличием ССЗ (1) и без ССЗ(0)",
    text_auto=True
    )

fig_heart.update_layout(bargap=0.2)
fig_heart.update_traces(marker_color=['rgb(0, 0, 100)', 'rgb(0, 200, 200)'], opacity=0.6)
fig_heart.update_traces(textfont_size=12, textangle=0, textposition="outside")
fig_heart.show()

## Поиск дубликатов

Выполним поиск явных дубликатов в датасете, а также неявных дубликатов для следующих столбцов с типом данных `object`:
1. `ChestPainType`
2. `RestingECG`
3. `ST_Slope`

Проверим исходный датасет на наличие явных дубликатов при помощи метода `.duplicated()`

In [12]:
print(data_initial.duplicated().sum())

0


Проверим исходный датасет на наличие явных дубликатов при помощи метода `.value_counts()`

In [13]:
data_initial['ChestPainType'].value_counts()

ASY    496
NAP    203
ATA    173
TA      46
Name: ChestPainType, dtype: int64

In [14]:
data_initial['RestingECG'].value_counts()

Normal    552
LVH       188
ST        178
Name: RestingECG, dtype: int64

In [15]:
data_initial['ST_Slope'].value_counts()

Flat    460
Up      395
Down     63
Name: ST_Slope, dtype: int64

Как мы видим, в датасете отсутствуют явные и неявные дубликаты.

## Поиск выбросов

Напишем функцию, которая подсчитывает количество выбросов по каждому столбцу и выводит их долю от общего числа данных в датасете. 

In [16]:
def outliers(df, column):
    initial_df = df
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    upper = Q3+1.5*IQR
    lower = Q1-1.5*IQR
    
    new_df = df.loc[(df[column] < lower) | (df[column] > upper)]
    
    initial = len(initial_df)
    new = len(new_df)
    ratio = (new/initial)*100
    
    print(
        "Столбец {}\n25-й квантиль = {:.0f},\n75-й квантиль = {:.0f},\n"\
        "Верхняя граница = {:.0f},\nНижняя граница = {:.0f},\nКоличество строк в исходном датасете = {:.0f},\n"\
        "Количество выбросов = {:.0f},\nДоля выбросов = {:.3f}\n".format(column,Q1,Q3,upper,lower,initial, new, ratio)
    )

In [17]:
for i in ["Age","RestingBP","Cholesterol","MaxHR","Oldpeak"]:
    outliers(data_initial, i)

Столбец Age
25-й квантиль = 47,
75-й квантиль = 60,
Верхняя граница = 80,
Нижняя граница = 28,
Количество строк в исходном датасете = 918,
Количество выбросов = 0,
Доля выбросов = 0.000

Столбец RestingBP
25-й квантиль = 120,
75-й квантиль = 140,
Верхняя граница = 170,
Нижняя граница = 90,
Количество строк в исходном датасете = 918,
Количество выбросов = 28,
Доля выбросов = 3.050

Столбец Cholesterol
25-й квантиль = 173,
75-й квантиль = 267,
Верхняя граница = 408,
Нижняя граница = 33,
Количество строк в исходном датасете = 918,
Количество выбросов = 183,
Доля выбросов = 19.935

Столбец MaxHR
25-й квантиль = 120,
75-й квантиль = 156,
Верхняя граница = 210,
Нижняя граница = 66,
Количество строк в исходном датасете = 918,
Количество выбросов = 2,
Доля выбросов = 0.218

Столбец Oldpeak
25-й квантиль = 0,
75-й квантиль = 2,
Верхняя граница = 4,
Нижняя граница = -2,
Количество строк в исходном датасете = 918,
Количество выбросов = 16,
Доля выбросов = 1.743



Как видно из полученного отчета, в датасете содержатся выброся, причем их доля доходит до 20% (`Cholesterol`)

## Выводы по результатам анализа данных

На данном этапе были выполнены следующие мероприятия:
1. Загружен и изучен исходный датасет.
2. Получено графическое представление о данных по отдельным полям датасета.
3. Проведен поиск аномалий, дубликатов и выбросов в датасете.

<b>Изучение исходного датасета</b>:
1. Целевой признак находится в поле `HeartDisease`.
2. Матрица корреляции по Пирсону показала, что целевой признак может иметь линейную зависимость со следующими полями: `MaxHR`,`ExerciseAngina`, `Oldpeak`.
3. В исходном датасете отсутствуют пропуски.

<b>Графический анализ</b>:
1. Графический анализ показал, что в датасете содержится примерно одинаковое содержание пациентов с ССЗ и без них.
2. В поле `Cholesterol` содержится значительное число пациентов с нулевый уровнем, что, скорее всего, является аномалией.
3. Пациенты с ССЗ чаще встречаются в более возрастных группах.
4. Показания частоты пульса у пациентов с ССЗ в среднем выше, чему пациентов без ССЗ.

<b>Поиск дубликатов, аномалий и выбросов</b>:
1. В исходном датасете отсутствуют явные и неявные дубликаты.
2. В датасете присутствуют выбросы, их доля доходит по 20% (`Cholesterol`).


<b>По результатам анадиза данных принято решение не удалять из датасета выбросы и аномалии и не менять их значения, так как изменение значений может привести к тому, что обученная модель после обучения будет плохо справляться с аномалиями и выбросами и, как следствие, предсказаниям такой модели нельзя будет доверять.</b>

# Подготовка данных для обучения

## Выбор метрики

В ходе исследования будет решаться задача бинарной классификации - модели необходимо будет определить, имеется ли у пациента СЗЗ (1) или нет(0).

Принимая во внимание, что модель будет предсказывать наличие или отсутствие болезни у пациента, крайне важно снизить количество случаев, когда пациент с СЗЗ принимается системой как здоровый, т.е. <b> снизить количество ложноотрицательных ответов </b>. Следовательно, в качестве целевой метрики принимаем полноту (`recall`).

## Прямое кодирование признаков

Для перевода признаков применим прямое кодирование, так как результаты прямого кодирования могут быть применены как при обучении моделей решающего дерева и случайного леса, так и для логистической регрессии.

Кроме того, разброс значений в количественных признаках может сказаться на результатах работы модели. Проведем масштабирование признаков для следующих столбцов:
1. `RestingBP`.
2. `Cholesterol`.
3. `MaxHR`.

Для прямого кодирования признаков воспользуемся функцией библиотеки Pandas `.get_dummies()`. При вызове функции также укажем аргумент `drop_first=True`, что позовлит избежать т.н. дамми-ловушки. 

In [18]:
data_ohe=pd.get_dummies(data_initial, drop_first=True)

data_ohe.head(3)

,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak,HeartDisease,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ST_Slope_Flat,ST_Slope_Up
0,40,1,140,289,0,172,0,0.0,0,1,0,0,1,0,0,1
1,49,0,160,180,0,156,0,1.0,1,0,1,0,1,0,1,0
2,37,1,130,283,0,98,0,0.0,0,1,0,0,0,1,0,1


## Масштабирование признаков

Для масштабирования признаков воспользуемся структурой `StandardScaler`, но для этого разделим выборку на обучающую и валидационную, а также выделим целевой признак. 

Ранее указанные для масштабирования столбцы сохраним в переменной `scale`.

In [19]:
target = data_ohe['HeartDisease']
features = data_ohe.drop('HeartDisease', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=data_ohe['HeartDisease'])

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)

scale = ['RestingBP', 'Cholesterol', 'MaxHR']

scaler = StandardScaler()
scaler.fit(features_train[scale])

features_train[scale] = scaler.transform(features_train[scale])
features_valid[scale] = scaler.transform(features_valid[scale])
features_test[scale] = scaler.transform(features_test[scale])

# Обучение моделей

Для обучения возьмем следующие модели:
1. Решающее дерево.
2. Случайный лес.
3. Логистическая регрессия.
4. Градиентный бустинг (Catboost).

Для подбора гиперпараметров применим кросс валидацию средствами библиотек `skleart` (GridSearchCV) и `optuna`.

## Решающее дерево

In [20]:
parameters = {
    'max_depth':[3,5,10,None],
    'max_features':[1,3,5,7],
    'min_samples_leaf':[1,2,3],
    'min_samples_split':[1,2,3]
}

grid_t = GridSearchCV(DecisionTreeClassifier(random_state=12345), parameters, scoring='recall', cv=5)
grid_t.fit(features_train, target_train)

grid_t.best_params_

{'max_depth': 5,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 2}

In [21]:
tree_1 = grid_t.best_score_
tree_1

0.8783060109289618

In [22]:
def objective(trial):

    parameters = {
        'max_depth':trial.suggest_int('max_depth', 1, 100),
        'max_features':trial.suggest_int('max_features', 1, 10),
        'min_samples_leaf':trial.suggest_int('min_samples_leaf', 2, 5),
        'min_samples_split':trial.suggest_int('min_samples_split', 2, 5)
    }

    model = DecisionTreeClassifier(**parameters)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)

    recall = recall_score(target_valid, predictions)

    return recall

In [23]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_trial.value)

tree_2 = study.best_trial.value

[I 2022-12-06 22:36:42,678] A new study created in memory with name: no-name-e9d335e7-529c-4fb7-bc34-2cfbc4dc06ed
[I 2022-12-06 22:36:42,691] Trial 0 finished with value: 0.7254901960784313 and parameters: {'max_depth': 47, 'max_features': 6, 'min_samples_leaf': 4, 'min_samples_split': 5}. Best is trial 0 with value: 0.7254901960784313.
[I 2022-12-06 22:36:42,701] Trial 1 finished with value: 0.8333333333333334 and parameters: {'max_depth': 57, 'max_features': 1, 'min_samples_leaf': 3, 'min_samples_split': 3}. Best is trial 1 with value: 0.8333333333333334.
[I 2022-12-06 22:36:42,709] Trial 2 finished with value: 0.7941176470588235 and parameters: {'max_depth': 4, 'max_features': 10, 'min_samples_leaf': 5, 'min_samples_split': 4}. Best is trial 1 with value: 0.8333333333333334.
[I 2022-12-06 22:36:42,717] Trial 3 finished with value: 0.8725490196078431 and parameters: {'max_depth': 19, 'max_features': 6, 'min_samples_leaf': 3, 'min_samples_split': 5}. Best is trial 3 with value: 0.8725

Number of finished trials: 10
Best trial: {'max_depth': 19, 'max_features': 6, 'min_samples_leaf': 3, 'min_samples_split': 5}
Best score: 0.8725490196078431


## Случайный лес

In [24]:
parameters = {'max_depth':[3,5,10],
              'n_estimators':[100,200],
              'max_features':[1,3,5],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }

grid_f = GridSearchCV(RandomForestClassifier(random_state=12345), parameters, scoring='recall', cv=5)
grid_f.fit(features_train, target_train)

grid_f.best_params_

{'max_depth': 5,
 'max_features': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 100}

In [25]:
forest_1 = grid_f.best_score_
forest_1

0.9145355191256831

In [26]:
def objective(trial):

    parameters = {
        'max_depth':trial.suggest_int('max_depth', 1, 100),
        'n_estimators':trial.suggest_int('n_estimators', 100, 1000),
        'max_features':trial.suggest_int('max_features', 1, 10),
        'min_samples_leaf':trial.suggest_int('min_samples_leaf', 2, 5),
        'min_samples_split':trial.suggest_int('min_samples_split', 2, 5)
    }

    model = RandomForestClassifier(**parameters)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)

    recall = recall_score(target_valid, predictions)

    return recall

In [27]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_trial.value)

forest_2 = study.best_trial.value

[I 2022-12-06 22:39:05,625] A new study created in memory with name: no-name-f1d2992e-17f9-47fe-a23c-dda79dde2c8d
[I 2022-12-06 22:39:07,202] Trial 0 finished with value: 0.9019607843137255 and parameters: {'max_depth': 38, 'n_estimators': 998, 'max_features': 1, 'min_samples_leaf': 2, 'min_samples_split': 4}. Best is trial 0 with value: 0.9019607843137255.
[I 2022-12-06 22:39:08,182] Trial 1 finished with value: 0.9019607843137255 and parameters: {'max_depth': 51, 'n_estimators': 654, 'max_features': 2, 'min_samples_leaf': 4, 'min_samples_split': 4}. Best is trial 0 with value: 0.9019607843137255.
[I 2022-12-06 22:39:09,169] Trial 2 finished with value: 0.8921568627450981 and parameters: {'max_depth': 79, 'n_estimators': 648, 'max_features': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}. Best is trial 0 with value: 0.9019607843137255.
[I 2022-12-06 22:39:10,364] Trial 3 finished with value: 0.8725490196078431 and parameters: {'max_depth': 55, 'n_estimators': 727, 'max_features': 6

Number of finished trials: 10
Best trial: {'max_depth': 38, 'n_estimators': 998, 'max_features': 1, 'min_samples_leaf': 2, 'min_samples_split': 4}
Best score: 0.9019607843137255


## Логистическая регрессия

In [28]:
parameters = {'penalty': ['l1','l2'],
             'C':[1, 10, 100, 1000]}

grid_r = GridSearchCV(LogisticRegression(random_state=12345), parameters, scoring='recall', cv=5)
grid_r.fit(features_train, target_train)

grid_r.best_params_

{'C': 1, 'penalty': 'l2'}

In [29]:
regr_1 = grid_r.best_score_
regr_1

0.8981420765027324

In [30]:
def objective(trial):

    parameters = {
        'penalty': 'l2',
        'C':trial.suggest_int('C', 1, 1000),
    }

    model = LogisticRegression(**parameters)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)

    recall = recall_score(target_valid, predictions)

    return recall

In [31]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_trial.value)

regr_2 = study.best_trial.value

[I 2022-12-06 22:39:17,823] A new study created in memory with name: no-name-e8b91ebb-8c23-4e10-9963-86f0cc7655f6
[I 2022-12-06 22:39:17,881] Trial 0 finished with value: 0.8823529411764706 and parameters: {'C': 575}. Best is trial 0 with value: 0.8823529411764706.
[I 2022-12-06 22:39:17,917] Trial 1 finished with value: 0.8823529411764706 and parameters: {'C': 540}. Best is trial 0 with value: 0.8823529411764706.
[I 2022-12-06 22:39:17,947] Trial 2 finished with value: 0.8823529411764706 and parameters: {'C': 370}. Best is trial 0 with value: 0.8823529411764706.
[I 2022-12-06 22:39:17,977] Trial 3 finished with value: 0.8823529411764706 and parameters: {'C': 987}. Best is trial 0 with value: 0.8823529411764706.
[I 2022-12-06 22:39:18,006] Trial 4 finished with value: 0.8823529411764706 and parameters: {'C': 104}. Best is trial 0 with value: 0.8823529411764706.
[I 2022-12-06 22:39:18,036] Trial 5 finished with value: 0.8725490196078431 and parameters: {'C': 331}. Best is trial 0 with v

Number of finished trials: 10
Best trial: {'C': 575}
Best score: 0.8823529411764706


## Градиентный бустинг

In [32]:
target = data_initial_cb['HeartDisease']
features = data_initial_cb.drop('HeartDisease', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=data_initial_cb['HeartDisease'])

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)

In [33]:
cat_features = [1,2,6,8,10]

In [34]:
parameters = {'iterations':[10,30,50],
              'random_seed':[10,30,50],
              'learning_rate':[0.1, 0.3, 0.5],
              'cat_features':[cat_features],
              'verbose': [False]
           }

grid_c = GridSearchCV(CatBoostClassifier(), parameters, scoring='recall', cv=5)
grid_c.fit(features_train, target_train)

grid_c.best_params_

{'cat_features': [1, 2, 6, 8, 10],
 'iterations': 10,
 'learning_rate': 0.3,
 'random_seed': 30,
 'verbose': False}

In [35]:
cat_1 = grid_c.best_score_

In [36]:
def objective(trial):

    parameters = {
        'iterations':trial.suggest_int('iterations', 1, 1000),
        'random_seed':trial.suggest_int('random_seed', 1, 1000),
        'learning_rate':trial.suggest_loguniform('learning_rate',0.1, 1),
        'verbose': trial.suggest_categorical('verbose',[False])
    }

    model = CatBoostClassifier(**parameters)
    model.fit(features_train, target_train, cat_features=cat_features)
    predictions = model.predict(features_test)

    recall = recall_score(target_test, predictions)

    return recall

In [37]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_trial.value)

cat_2 = study.best_trial.value

[I 2022-12-06 22:39:22,952] A new study created in memory with name: no-name-7e58ff3c-61ad-4fb4-8e49-878c5decaaf0
[I 2022-12-06 22:39:23,846] Trial 0 finished with value: 0.8921568627450981 and parameters: {'iterations': 629, 'random_seed': 748, 'learning_rate': 0.5842986202709064, 'verbose': False}. Best is trial 0 with value: 0.8921568627450981.
[I 2022-12-06 22:39:24,589] Trial 1 finished with value: 0.9215686274509803 and parameters: {'iterations': 472, 'random_seed': 787, 'learning_rate': 0.10147815352107088, 'verbose': False}. Best is trial 1 with value: 0.9215686274509803.
[I 2022-12-06 22:39:25,329] Trial 2 finished with value: 0.9215686274509803 and parameters: {'iterations': 535, 'random_seed': 457, 'learning_rate': 0.8521184762132826, 'verbose': False}. Best is trial 1 with value: 0.9215686274509803.
[I 2022-12-06 22:39:25,442] Trial 3 finished with value: 0.9215686274509803 and parameters: {'iterations': 106, 'random_seed': 283, 'learning_rate': 0.41728179629312484, 'verbos

Number of finished trials: 10
Best trial: {'iterations': 157, 'random_seed': 26, 'learning_rate': 0.7607266723443574, 'verbose': False}
Best score: 0.9509803921568627


## Сравнение результатов обучения моделей

In [38]:

fig = ff.create_annotated_heatmap(
        z=[
            [float("{:.3f}".format(tree_1)), float("{:.3f}".format(forest_1)), float("{:.3f}".format(regr_1)), float("{:.3f}".format(cat_1))],
            [float("{:.3f}".format(tree_2)), float("{:.3f}".format(forest_2)), float("{:.3f}".format(regr_2)), float("{:.3f}".format(cat_2))]],
        x=['Decision Tree','Random Forest', 'Logistic Regression', 'CatBoost'],
        y=['GridSearchCV','Optuna'],       
        colorscale='ylgnbu',
        colorbar_thickness=30,
        colorbar_ticklen=3
    )
fig.update_layout(title_text='<b>Recall Value<b>',
                    title_x=0.5,
                    titlefont={'size': 24},
                    width=600, height=450,
                    xaxis_showgrid=False,
                    xaxis={
                        'title': 'Models',
                        'side': 'bottom'
                        },
                    yaxis_showgrid=False,
                    yaxis={
                        'title': 'Cross Validation',
                        'side': 'bottom'
                        },
                    yaxis_autorange='reversed',                   
                    paper_bgcolor=None,
    )

fig.show()

По результатам обучения моделей и подбора гиперпараметров наибольшие результаты получены для модели <b>Catboost</b> с (0.95)

## Получение предсказаний модели

Обучим модель, выбранную по результатам кросс-валидации, получим предсказания и посчитаем для нее значение выбранной метрики.

In [44]:
model = CatBoostClassifier(
    iterations=157,
    random_seed=26,
    learning_rate=0.7607266723443574,
    verbose=False,
    custom_loss='Recall'
)

model.fit(
    features_train, target_train,
    cat_features=cat_features,
    eval_set=(features_valid, target_valid),
    verbose=False,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [45]:
report = classification_report(target_test, model.predict(features_test), target_names=['Without_HeartDisease', 'With_HeartDisease'])
print(report)

                      precision    recall  f1-score   support

Without_HeartDisease       0.93      0.83      0.88        82
   With_HeartDisease       0.87      0.95      0.91       102

            accuracy                           0.90       184
           macro avg       0.90      0.89      0.89       184
        weighted avg       0.90      0.90      0.90       184



In [46]:
conf_m = confusion_matrix(target_test, model.predict(features_test))

fig = ff.create_annotated_heatmap(
        z=conf_m,
        x=['Without_HeartDisease','With_HeartDisease'],
        y=['Without_HeartDisease','With_HeartDisease'],       
        colorscale='ylgnbu',
        colorbar_thickness=30,
        colorbar_ticklen=3,
    )
fig.update_layout(title_text='<b>Confusion matrix<b>',
                    title_x=0.5,
                    titlefont={'size': 24},
                    width=600, height=550,
                    xaxis_showgrid=False,
                    xaxis={
                        'title': 'Predicted values',
                        'side': 'bottom'
                        },
                    yaxis_showgrid=False,
                    yaxis={
                        'title': 'True values',
                        'side': 'bottom'
                        },
                    yaxis_autorange='reversed',                   
                    paper_bgcolor=None,
    )

fig.show()

In [47]:
importances = pd.DataFrame(model.feature_importances_,
                         features_test.columns)

In [48]:
fig_feat = px.bar(
    importances,
    x=importances[0],
    )

fig_feat.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig_feat.update_xaxes(title_text='Importance')
fig_feat.update_yaxes(title_text='Feature')
fig_feat.update_layout(title_text='Features importance')
fig_feat.update_traces(marker_color='rgb(0, 200, 200)', opacity=0.6)

fig_feat.show()

# Выводы

По результатам выполненной работы можно сделать следующие выводы:
1. Все выбранные модели показывают достаточно высокие значения метрики recall при любом методе подбора гиперпараметров (от 0.873 до 0.951).
2. Наибольшее значение полноты достигнуто при использовании градиентного бустинга Catboost - 0.95.
3. График зависимости свойств показал сильную зависимость от следующих признаков: `Cholesterol` и `ChestpainType`.

<b>В целом следует отметить, что исходный датасет имел всего 1000 значений, и при разделении его на 3 выборки - обучающую, валидационную и тестовую в соотношении 60:20:20 - в нашем распоряжении остается 200 значений для валидации и 200 значений для предсказаний и, следовательно, даже 1-2 дополнительных ложноотрицательных значения приводят с ощутимому изменению выбранной метрики.</b>